In [ ]:
# Final Assignment: Extracting and Visualizing Stock Data
# Clean Solutions for Questions 1–6

# --- Import Required Libraries ---
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- Helper Function to Plot Graphs and Save ---
def make_graph(stock_data, revenue_data, stock_name, filename=None):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        subplot_titles=("Historical Share Price", "Historical Revenue"),
                        vertical_spacing=0.3)
    
    fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Close'],
                             name="Share Price"), row=1, col=1)
    
    fig.add_trace(go.Scatter(x=revenue_data['Date'], y=revenue_data['Revenue'],
                             name="Revenue"), row=2, col=1)
    
    fig.update_layout(showlegend=False,
                      height=900,
                      title=stock_name,
                      xaxis_rangeslider_visible=True)
    
    fig.show()
    
    # Save plots if filename is provided
    if filename:
        fig.write_image(f"{filename}.png")
        fig.write_html(f"{filename}.html")

# --- Question 1: Use yfinance to Extract Tesla Stock Data ---
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)

# --- Question 2: Use Webscraping to Extract Tesla Revenue Data ---
url_tesla = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_tesla = requests.get(url_tesla).text
soup_tesla = BeautifulSoup(html_tesla, "html.parser")
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in soup_tesla.find_all("tr"):
    cols = row.find_all("td")
    if len(cols) == 2:
        date = cols[0].text.strip()
        revenue = cols[1].text.strip().replace(",", "").replace("$", "")
        if revenue != "":
            tesla_revenue = tesla_revenue.append({"Date": date, "Revenue": revenue}, ignore_index=True)

tesla_revenue["Revenue"] = pd.to_numeric(tesla_revenue["Revenue"], errors="coerce")
tesla_revenue.dropna(inplace=True)

# --- Question 3: Use yfinance to Extract GME Stock Data ---
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)

# --- Question 4: Use Webscraping to Extract GME Revenue Data ---
url_gme = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
html_gme = requests.get(url_gme).text
soup_gme = BeautifulSoup(html_gme, "html.parser")
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in soup_gme.find_all("tr"):
    cols = row.find_all("td")
    if len(cols) == 2:
        date = cols[0].text.strip()
        revenue = cols[1].text.strip().replace(",", "").replace("$", "")
        if revenue != "":
            gme_revenue = gme_revenue.append({"Date": date, "Revenue": revenue}, ignore_index=True)

gme_revenue["Revenue"] = pd.to_numeric(gme_revenue["Revenue"], errors="coerce")
gme_revenue.dropna(inplace=True)

# --- Question 5: Plot Tesla Stock Graph ---
make_graph(tesla_data.set_index("Date"), tesla_revenue, "Tesla", filename="tesla_stock")

# --- Question 6: Plot GameStop Stock Graph ---
make_graph(gme_data.set_index("Date"), gme_revenue, "GameStop", filename="gamestop_stock")

# --- Save Cleaned Datasets to CSV ---
tesla_data.to_csv("tesla_stock_data.csv", index=False)
tesla_revenue.to_csv("tesla_revenue_data.csv", index=False)
gme_data.to_csv("gamestop_stock_data.csv", index=False)
gme_revenue.to_csv("gamestop_revenue_data.csv", index=False)
